In [1]:
from pycaret.classification import *
from selenium import webdriver
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
import random
from datetime import datetime, timedelta
import pickle
from fuzzywuzzy import fuzz, process
from fractions import Fraction
import time
import collections
import concurrent.futures
from collections import defaultdict
import json
from oddsapi import OddsApiClient
import requests
from bs4 import BeautifulSoup

C:\Users\idotz\anaconda3\envs\p37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\idotz\anaconda3\envs\p37\lib\site-packages\numpy\.libs\libopenblas.4SP5SUA7CBGXUEOC35YP2ASOICYYEQZZ.gfortran-win_amd64.dll
C:\Users\idotz\anaconda3\envs\p37\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [22]:
open_file = open('model_columns.pkl', "rb")
model_columns = list(pickle.load(open_file))
open_file.close()

In [27]:
model_columns

['Shot Attempts',
 'Total xG',
 'Offsides',
 'Pass Completion',
 'Goals',
 'Corners',
 'Fouls Committed',
 'Total Shots on Target',
 'Shots Blocked']

In [33]:
scraped_cols = ['Total shots','Expected goals (xG)','Offsides','Accurate passes percentage','score','Corners','Fouls committed','Shots on target','Blocks']
pred_cols = dict(zip(model_columns, scraped_cols))

In [82]:
pred_cols

{'Shot Attempts': 'Total shots',
 'Total xG': 'Expected goals (xG)',
 'Offsides': 'Offsides',
 'Pass Completion': 'Accurate passes percentage',
 'Goals': 'score',
 'Corners': 'Corners',
 'Fouls Committed': 'Fouls committed',
 'Total Shots on Target': 'Shots on target',
 'Shots Blocked': 'Blocks'}

In [37]:
model = load_model('lr_model')

Transformation Pipeline and Model Successfully Loaded


In [108]:
today = f"{datetime.now():%Y%m%d}"
game_data = fotmob_game_data(f'https://www.fotmob.com/?date={today}')
all_odds = get_all_odds_links()
days_data = connect_links(game_data, all_odds)

{'2022-08-26 13:53:00': [['horsens-vs-agf-aarhus', 'DENMARK - SUPERLIGAEN', 'https://www.fotmob.com/match/3887412/matchfacts/horsens-vs-agf-aarhus', 'https://www.oddschecker.com/football/denmark/superligaen/ac-horsens-v-agf-aarhus/winner']], '2022-08-26 14:53:00': [['girona-vs-celta-vigo', 'SPAIN - LALIGA', 'https://www.fotmob.com/match/3917956/matchfacts/girona-vs-celta-vigo', 'https://www.oddschecker.com/football/spain/la-liga-primera/girona-v-celta-vigo/winner'], ['nec-nijmegen-vs-fc-groningen', 'NETHERLANDS - EREDIVISIE', 'https://www.fotmob.com/match/3900368/matchfacts/nec-nijmegen-vs-fc-groningen', 'https://www.oddschecker.com/football/netherlands/eredivisie/nec-v-fc-groningen/winner']], '2022-08-26 15:23:00': [['sc-freiburg-vs-vfl-bochum', 'GERMANY - BUNDESLIGA', 'https://www.fotmob.com/match/3903573/matchfacts/sc-freiburg-vs-vfl-bochum', 'https://www.oddschecker.com/football/germany/bundesliga/sc-freiburg-v-bochum/winner']], '2022-08-26 15:38:00': [['sporting-charleroi-vs-club-

In [127]:
days_data

{'2022-08-26 13:53:00': [['horsens-vs-agf-aarhus',
   'DENMARK - SUPERLIGAEN',
   'https://www.fotmob.com/match/3887412/matchfacts/horsens-vs-agf-aarhus',
   'https://www.oddschecker.com/football/denmark/superligaen/ac-horsens-v-agf-aarhus/winner']],
 '2022-08-26 14:53:00': [['girona-vs-celta-vigo',
   'SPAIN - LALIGA',
   'https://www.fotmob.com/match/3917956/matchfacts/girona-vs-celta-vigo',
   'https://www.oddschecker.com/football/spain/la-liga-primera/girona-v-celta-vigo/winner'],
  ['nec-nijmegen-vs-fc-groningen',
   'NETHERLANDS - EREDIVISIE',
   'https://www.fotmob.com/match/3900368/matchfacts/nec-nijmegen-vs-fc-groningen',
   'https://www.oddschecker.com/football/netherlands/eredivisie/nec-v-fc-groningen/winner']],
 '2022-08-26 15:23:00': [['sc-freiburg-vs-vfl-bochum',
   'GERMANY - BUNDESLIGA',
   'https://www.fotmob.com/match/3903573/matchfacts/sc-freiburg-vs-vfl-bochum',
   'https://www.oddschecker.com/football/germany/bundesliga/sc-freiburg-v-bochum/winner']],
 '2022-08-26 

In [ ]:
c = days_data.copy()
todays_data = []

while True:
    for clock in c.copy().keys():
        if datetime.strptime(clock, "%Y-%m-%d %H:%M:%S") <= datetime.now():
            stats_urls = [x[2] for x in c[clock]]
            odds_urls = [x[3] for x in c[clock]]
            print(stats_urls, odds_urls)
            with concurrent.futures.ThreadPoolExecutor() as executor:
                r = executor.map(stats_odds, stats_urls, odds_urls)
            for result in r:
                todays_data.append(result)
            del c[clock]
    print(f'{len(c)} halftimes left -- {len(todays_data)}\n')
    if len(c) == 0:
        print('all done')
        break
    time.sleep(60)

open_file = open(f'{today}_data.pkl', "wb")
pickle.dump(today_data, open_file)
open_file.close()

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes left -- 0

10 halftimes lef

In [20]:
t = {'Expected goals (xG)': -0.030000000000000027,
  'Total shots': 2.0,
  'Big chances': -1.0,
  'Big chances missed': -1.0,
  'Accurate passes': 119.0,
  'Accurate passes percentage': 0.12,
  'Fouls committed': 0.0,
  'Offsides': 0.0,
  'Corners': 1.0,
  'Shots off target': 0.0,
  'Shots on target': -3.0,
  'Blocked shots': 5.0,
  'Hit woodwork': 0.0,
  'Shots inside box': 1.0,
  'Shots outside box': 1.0,
  'xG open play': -0.26,
  'xG set play': 0.24,
  'xG on target (xGOT)': -0.72,
  'Passes': 120.0,
  'Own half': 54.0,
  'Opposition half': 65.0,
  'Accurate long balls': 19.0,
  'Accurate long balls percentage': 0.39,
  'Accurate crosses': 2.0,
  'Accurate crosses percentage': -0.09999999999999998,
  'Throws': 0.0,
  'Yellow cards': -1.0,
  'Red cards': 0.0,
  'Tackles won': 5.0,
  'Tackles won percentage': 0.22000000000000003,
  'Interceptions': -2.0,
  'Blocks': -5.0,
  'Clearances': -10.0,
  'Keeper saves': 1.0,
  'Duels won': 1.0,
  'Ground duels won': 2.0,
  'Ground duels won percentage': 0.10000000000000003,
  'Aerial duels won': -1.0,
  'Aerial duels won percentage': -0.06999999999999995,
  'Successful dribbles': -5.0,
  'Successful dribbles percentage': 0.18000000000000005,
  'score': -3.0,
  'posession': 32.0,
  'match_name': 'leon-vs-mazatlan-fc',
  'half_time': 'Half-Time',
  'url': 'https://www.fotmob.com/match/3888521/matchfacts/leon-vs-mazatlan-fc',
     'odds': {'Club Leon': 36.0,
  'Draw': 13.0,
  'Mazatlan FC': 1.05}}

In [92]:
def predicted_probs(results):
    d = {}
    for k, v in results.items():
        try:
            d[k] = float(v[0]) - float(v[1])
        except:
            d[k] = v
    for_pred = [[d[key] for key in pred_cols.values()]]
    pred = model[-1].predict_proba(for_pred)
    return pred

In [89]:
x = get_stats('https://www.fotmob.com/match/3900956/matchfacts/leeds-united-vs-chelsea')

not halftime just yet ... sleeping
time: Full-Time


In [38]:
def make_bet(decimal_odds, predicted_prob, k_weight):
    if 1/decimal_odds >= predicted_prob:
        return 'no value'
    else:
        return kelly(predicted_prob, decimal_odds, k_weight)

In [130]:
def print_advice(results):
    try:
        p_probs = predicted_probs(results)[0]
        predicted = np.argmax(p_probs)
        decimal_odds = list(results['odds'].values())
        bet_made = {}
        print('\033[1m' + results['match_name'] + '\033[0m')
        print(f'Predicted winner is {predicted}\n')
        for p_prob, d_odd, win in zip(p_probs, decimal_odds, ['draw', 'home','away']):
            p_prob = round(p_prob, 3)
            d_odd = round(d_odd, 3)
            size = make_bet(d_odd, p_prob, 0.5)
            print(f'{win}: predicted prob: {p_prob*100}% - oddschecker: {round(1/d_odd*100,2)}% ({d_odd}) ---- bet {size}')
            bet_made[win] = size
        print('\n')
        bet_made['odds_checker_sum'] = np.sum([1/o for o in decimal_odds])
        bet_made['winner'] = predicted
        bet_made['predicted_probs'] = np.round(p_probs, 3)
        bets = {'bet':bet_made} 
        row = {**results, **bets}
    except:
        row = results
    return row

In [133]:
print(print_advice('hello'))

hello


In [126]:
z = print_advice(t)

leon-vs-mazatlan-fc
Predicted winner is 2

draw: predicted prob: 9.8% - oddschecker: 2.78% (36.0) ---- bet 0.0361
home: predicted prob: 0.3% - oddschecker: 7.69% (13.0) ---- bet no value
away: predicted prob: 89.9% - oddschecker: 95.24% (1.05) ---- bet no value




In [41]:
def kelly(p, b, weight):
    stake = round((p * (b - 1) - (1 - p)) / (b - 1), 4) * weight
    return stake if stake > 0.0 else 'no value'

In [42]:
def getname(name, l):
    if len(l) == 0:
        return ''
    highest = process.extractOne(name,l)
    return highest[0]

In [136]:
def get_stats(url):
    driver = webdriver.Chrome()
    try:
        driver.implicitly_wait(random.randint(0, 6))
        driver.get(url)
        ht = driver.find_elements(By.XPATH, '//span[@class="css-8ngw4i-bottomRow"]')[1].text
        if ht != 'Half-Time':
            print('not halftime just yet ... sleeping\n')
            time.sleep(3*60)
            ht = driver.find_elements(By.XPATH, '//span[@class="css-8ngw4i-bottomRow"]')[1].text
        button = driver.find_elements(By.XPATH, '(//button[@class="css-12l9xvx-ExpandButton e7i6a0l0"])[2]')[0]
        button.click()
        div1 = driver.find_elements(By.XPATH, '//li[@class="css-radwzz-Stat e683amr6"]')
        t = {k[1]: [k[0], k[2]] for k in [i.split('\n') for i in [x for x in [d.text for d in div1]
              if x not in ['Shots','Passes','Expected goals (xG)','Discipline','Defence','Duels']]]}
        score = driver.find_element(By.XPATH, '//span[@class="css-bw7eig-topRow"]').get_attribute('textContent').split(' ')
        t['score'] = [score[0], score[2]]
        posession = driver.find_element(By.XPATH, '//div[@class="css-7s52se-PossessionWheel e683amr3"]').text.split('\n')
        t['posession'] = [posession[1].replace('%',''), posession[2].replace('%','')]
        game_stats = {}
        for name, stats in t.items():
            if '(' in ''.join(stats):
                s = stats[0].split(' ') + stats[1].split(' ')
                if len(s) == 4:
                    s = [i.replace('(', '').replace(')','').replace('%','') for i in s]
                    game_stats[name] = [s[0], s[2]]
                    game_stats[name + ' percentage'] = [int(s[1])/100, int(s[3])/100]
                else:
                    s = [i for i in s if '%' not in i]
                    game_stats[name] = s
            else:
                game_stats[name] = stats
#         game_stats = {k: float(v[0]) - float(v[1]) for k, v in game_stats.items()}
        game_stats['match_name'] = url.split('/')[-1]
        game_stats['half_time'] = ht
        game_stats['url'] = url
    except:
        game_stats = {}
    driver.quit()
    return game_stats

In [44]:
xg_leagues = {'UNITED STATES - MAJOR LEAGUE SOCCER':'world/usa/mls','ENGLAND - CHAMPIONSHIP':'english/championship',
              'ENGLAND - PREMIER LEAGUE': 'english/premier-league', 'GERMANY - BUNDESLIGA': 'germany/bundesliga',
              'FRANCE - LIGUE 1': 'france/ligue-1','AUSTRIA - BUNDESLIGA': 'austria/bundesliga',
              'PORTUGAL - LIGA PORTUGAL':'portugal/primeira-liga', 'BELGIUM - FIRST DIVISION A':'belgium/jupiler-pro-league',
              'SWITZERLAND - SUPER LEAGUE':'switzerland/super-league','SPAIN - LALIGA':'spain/la-liga-primera',
             'DENMARK - SUPERLIGAEN':'denmark/superligaen', 'MEXICO - LIGA MX APERTURA':'world/mexico/liga-mx',
             'NETHERLANDS - EREDIVISIE':'netherlands/eredivisie', 'SCOTLAND - PREMIERSHIP': 'scottish/premiership'}

def fotmob_game_data(url):
    driver = webdriver.Chrome()
    driver.implicitly_wait(3)
    driver.get(url)
    divs = driver.find_elements(By.XPATH, '//div[@class="css-aoqfcq-Group e7pc1841"]')
    today = url[-8:]
    league = {}
    for count, bucket in enumerate(divs):
        league_name = bucket.find_elements(By.XPATH, '//a[@class="css-vfisw6-GroupTitleLink e7pc1842"]')[count].text
        if league_name not in xg_leagues.keys():
            continue
        url = bucket.find_elements(By.TAG_NAME, 'a')
        urls = [u.get_attribute('href') for u in url]
        time = bucket.text
        time = [datetime.strptime(i + today, '%H:%M%Y%m%d') + timedelta(minutes=53) for i in time.split('\n') if ':' in i]
        urls = urls[1:]
        names = [u.split('/')[-1] for u in urls]
        league[league_name] = {n: [u,t] for u, t, n in zip(urls, time, names)}
    return league

In [46]:
def get_all_odds_links():
    odds_links = {}
    for f_league, url_league in xg_leagues.items():
        driver = webdriver.Chrome()
        driver.implicitly_wait(3)
        url = 'https://www.oddschecker.com/football/' + url_league
        driver.get(url)
        driver.implicitly_wait(3)
        divs = driver.find_elements(By.XPATH, '//a[@class="beta-callout full-height-link whole-row-link"]')
        mt = []
        for div in divs:
            odds_url = div.get_attribute('href')
            mt.append(odds_url)
        odds_links[f_league] = mt
    return odds_links

In [47]:
def connect_links(data, all_odds):
    for l in data.keys():
        fotmob_names = data[l].keys()
        odds_names = {v: k for k, v in zip(all_odds[l],[str(i).split('/')[-2] for i in all_odds[l]])}
        for game in fotmob_names:
            match = getname(game, odds_names.keys())
            try:
                link = odds_names[match]
            except:
                link = ''
            data[l][game] = data[l][game] + [link]
    scraper_data = collections.defaultdict(list)
    for l_name, data in data.items():
        for game_name, l1_t_l2 in data.items():
            scraper_data[str(l1_t_l2[1])].append([game_name, l_name, l1_t_l2[0], l1_t_l2[2]])
    scraper_data = dict(sorted(scraper_data.items(), key=lambda x: x[0]))
    return scraper_data

In [137]:
def get_odds(url):
    try:
        driver = webdriver.Chrome()
        if url == '':
            return {'url':url, odds: ''}
        driver.implicitly_wait(1)
        url = url
        driver.get(url)
        odds = driver.find_element(By.XPATH, '//div[@class="ExpanderBestOddsWrapper_e1jc6hcf"]').text.split('\n')
        odds = [float(Fraction(j)) + 1.0 if '/' in j else j for j in odds]
        odds = dict(zip(odds[::2], odds[1::2]))
        odds = {'url':url, 'odds': odds}
    except:
        odds = {'url':url, 'odd': ''}
    driver.quit()
    return odds

In [134]:
def stats_odds(stats_url, odds_url):
    stats = get_stats(stats_url)
    time.sleep(30)
    odds = get_odds(odds_url)
    row = {**stats, **odds}
    result = print_advice(row)
    return result

In [135]:
def get_odds(url):
    try:
        driver = webdriver.Chrome()
        if url == '':
            return {'url':url, odds: ''}
        driver.implicitly_wait(1)
        url = url
        driver.get(url)
        odds = driver.find_element(By.XPATH, '//div[@class="ExpanderBestOddsWrapper_e1jc6hcf"]').text.split('\n')
        odds = [float(Fraction(j)) + 1.0 if '/' in j else j for j in odds]
        odds = dict(zip(odds[::2], odds[1::2]))
        odds = {'url':url, 'odds': odds}
    except:
        odds = {'url':url, 'odd': ''}
    driver.quit()
    return odds